In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [7]:
#j'appelle le mot à rechercher à l'aide la fonction input
word = input("Tapez un mot :")

url_lily = f'https://www.thelily.com/tag/{word}/'
url_nylon = f'https://nylon.com/search/?q={word}'

html_lily = BeautifulSoup(requests.get(url_lily).content)
nylon_html=BeautifulSoup(requests.get(url_nylon).content)

Tapez un mot :love


In [8]:
#travail sur Lily News
lily_card=html_lily.find_all('div', class_='card-content align-items-start flex-container-row justify-space-between flex-mobile-column')

lily_article =[]
for x in lily_card:
    a=x.find_all('a')
    h3 = x.find_all('h3')
    lily_article.append([a,h3])

In [9]:
df_lily = pd.DataFrame(lily_article)
df_lily.columns=['titre','auteur']
df_lily['theme']=df_lily['titre']

In [10]:
def cleandf(a):
    return '|'.join([x.text.strip() for x in a])

dfcol=['titre','auteur']

for col in dfcol: 
    df_lily[col]=list(map(cleandf, df_lily[col]))

In [11]:
def linkdf(b):   
    return '|'.join([x.get('href') for x in b])

df_lily['theme']=list(map(linkdf, df_lily['theme']))

In [12]:
link = lambda a:'https://www.thelily.com'+str(a)
df_lily['theme']=list(map(link,df_lily['theme']))

In [13]:
df_lily['link'] = df_lily['theme'].str.split('|').str[0] 
df_lily['theme'] = df_lily['titre'].str.split('|').str[1]
df_lily['titre'] = df_lily['titre'].str.split('|').str[2] 
df_lily['accroche'] = df_lily['auteur'].str.split('|').str[0] 
df_lily['date'] = df_lily['auteur'].str.split('|').str[2] 
df_lily['auteur'] = df_lily['auteur'].str.split('|').str[1]
df_lily['source'] = 'The Lily'

In [14]:
df_lily = df_lily[['source', 'theme','titre','auteur','accroche','date','link',]]
df_lily.head(3)

,source,theme,titre,auteur,accroche,date,link
0,The Lily,Books,This romance novelist with Asperger’s writes a...,The Lily News,Helen Hoang’s novels have resonated with reade...,July 23,https://www.thelily.com/this-romance-novelist-...
1,The Lily,Movies,"At Rom Com fest, these fans still believe in h...",The Lily News,"Miraya Berke, the festival’s founder, curated ...",July 5,https://www.thelily.com/at-rom-com-fest-these-...
2,The Lily,History,It’s possible we had a gay president — but we ...,The Lily News,"Rose Cleveland, the sister of Grover Cleveland...",June 23,https://www.thelily.com/its-possible-we-had-a-...


In [15]:
#travail sur Nylon
nylon_card=nylon_html.find_all('div', class_='widget')
card = [x.find_all('article') for x in nylon_card]

nylon_article=[y.find_all(['a','span'])for x in card for y in x]

df_nylon = pd.DataFrame(nylon_article)
df_nylon = df_nylon.rename(columns={0: 'theme',1:'link',2:'titre',3:'auteur',4:'date'})
df_nylon['source']='nylon'
df_nylon['accroche']=''

In [16]:
dfcol=['theme','titre','auteur','date']
for col in dfcol:
    df_nylon[col]=([x.text.strip() for x in df_nylon[col]])

df_nylon['link'] = [x.get('href') for x in df_nylon['link']]
df_nylon = df_nylon[['source','theme','titre','auteur','accroche','date','link']]

In [17]:
df_nylon.head(3)

,source,theme,titre,auteur,accroche,date,link
0,nylon,TV,'Love Island' US Has Hooked Me With The Promis...,Sesali Bowen,,10 July,https://nylon.com/love-island-us-black-couple
1,nylon,Music,Courtney Love Claims Kurt Cobain's Ghost Spoke...,Bailey Calfee,,13 June,https://nylon.com/courtney-love-kurt-cobain-ghost
2,nylon,Music,"Terror Jr Are Huge ""Losers"" And They Love It",Bailey Calfee,,07 June,https://nylon.com/terror-jr-interview-video


In [18]:
df_search_word = df_lily.append(df_nylon)
df_search_word

,source,theme,titre,auteur,accroche,date,link
0,The Lily,Books,This romance novelist with Asperger’s writes a...,The Lily News,Helen Hoang’s novels have resonated with reade...,July 23,https://www.thelily.com/this-romance-novelist-...
1,The Lily,Movies,"At Rom Com fest, these fans still believe in h...",The Lily News,"Miraya Berke, the festival’s founder, curated ...",July 5,https://www.thelily.com/at-rom-com-fest-these-...
2,The Lily,History,It’s possible we had a gay president — but we ...,The Lily News,"Rose Cleveland, the sister of Grover Cleveland...",June 23,https://www.thelily.com/its-possible-we-had-a-...
3,The Lily,Weddings,How to put a lifetime of love into words,Caroline Kitchener,Five women on how they wrote their wedding vow...,May 23,https://www.thelily.com/how-to-put-a-lifetime-...
4,The Lily,Weddings,It’s okay to hate your wedding day,Caroline Kitchener,"Since childhood, women have been told that thi...",May 10,https://www.thelily.com/its-okay-to-hate-your-...
5,The Lily,Books,You’ll want to devour Sally Rooney’s sophomore...,Lauren Sarazen,"Youth, love and cowardice are unavoidably inte...",April 20,https://www.thelily.com/youll-want-to-devour-s...
6,The Lily,Technology,This app claims it can improve your marriage. ...,Caroline Kitchener,‘Remind me to express appreciation at 9 a.m.’ ...,April 12,https://www.thelily.com/this-app-claims-it-can...
7,The Lily,The Goodbye,"I am saying goodbye to Oahu, Hawaii. Here are ...",Jessie Wright,"After four years on the mythic island, I got w...",April 10,https://www.thelily.com/i-am-saying-goodbye-to...
8,The Lily,Relationships,More than half of young Americans don’t have a...,The Lily News,Fifty-one percent of those between ages 18 and...,March 22,https://www.thelily.com/more-than-half-of-youn...
9,The Lily,Love,‘Love is a spectrum’: 8 stories that go beyond...,Nneka McGuire,"This Valentine’s Day, we’re highlighting versi...",February 14,https://www.thelily.com/love-is-a-spectrum-8-s...


In [20]:
df_search_word.to_json('search_word_result.json', orient='records')